# Arizona Housing Market Dashboard Data

##### Goal: Analyze Arizona's largest real-estate markets.

#### Steps:
        1. Extract Redfin Data
        2. Transform tables
        3. Load to Power BI

# 1. Extracting Data from Redfin

In [2]:
import pandas as pd

#Redfin Data Source - https://www.redfin.com/news/data-center/
url = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/zip_code_market_tracker.tsv000.gz'

#Retrieving entire redfin data set
redfin_data_all_states = pd.read_csv(url, compression='gzip', sep='\t')


### 1.1 Mapping Redfin Zip Codes to U.S. Cities

In [3]:
#Creating a new table to map zip codes to county/city
mapping_df = pd.read_csv('https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv')

mapping_df  = (mapping_df
                        .assign(
                            zipcode = mapping_df['zipcode'].loc[mapping_df['zipcode'].str.isnumeric()].astype(int))
                        .drop(columns=['state_fips'])
                        .dropna()
                )

# 2. Transforming Tables (slicing, cleaning, merging)

In [4]:
#Arizona housing data
az_redfin_df = (redfin_data_all_states
                .loc[redfin_data_all_states['state_code'] == 'AZ']
                .assign(
                    zipcode = redfin_data_all_states['region'].str.replace('Zip Code:','').astype(int),
                    median_dom_mom = redfin_data_all_states.sort_values(by = ['period_begin'])['median_dom'].pct_change()
                    )
                .merge(mapping_df, on = 'zipcode', how = 'inner')
                .drop(columns = ['region','city_x','state_x'])
                .rename(columns = {'city_y':'city','state_y':'state'})
                )


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,state_code,property_type,property_type_id,...,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,zipcode,state,state_abbr,county,city
0,2018-06-01,2018-08-31,90,zip code,2,36530,f,AZ,All Residential,-1,...,0.052533,-0.040724,"Tucson, AZ",46060,2022-07-10 14:38:35,85737,Arizona,AZ,Pima,Oro valley
1,2019-12-01,2020-02-29,90,zip code,2,36530,f,AZ,Condo/Co-op,3,...,0.000000,-0.500000,"Tucson, AZ",46060,2022-07-10 14:38:35,85737,Arizona,AZ,Pima,Oro valley
2,2021-11-01,2022-01-31,90,zip code,2,36530,f,AZ,Townhouse,13,...,-0.666667,0.000000,"Tucson, AZ",46060,2022-07-10 14:38:35,85737,Arizona,AZ,Pima,Oro valley
3,2014-10-01,2014-12-31,90,zip code,2,36530,f,AZ,Townhouse,13,...,-0.500000,0.000000,"Tucson, AZ",46060,2022-07-10 14:38:35,85737,Arizona,AZ,Pima,Oro valley
4,2018-04-01,2018-06-30,90,zip code,2,36530,f,AZ,Townhouse,13,...,0.000000,-0.250000,"Tucson, AZ",46060,2022-07-10 14:38:35,85737,Arizona,AZ,Pima,Oro valley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92495,2019-03-01,2019-05-31,90,zip code,2,36283,f,AZ,All Residential,-1,...,NaN,NaN,"Phoenix, AZ",38060,2022-07-10 14:38:35,85232,Arizona,AZ,Pinal,Florence
92496,2019-01-01,2019-03-31,90,zip code,2,36283,f,AZ,Single Family Residential,6,...,NaN,NaN,"Phoenix, AZ",38060,2022-07-10 14:38:35,85232,Arizona,AZ,Pinal,Florence
92497,2019-01-01,2019-03-31,90,zip code,2,36283,f,AZ,All Residential,-1,...,NaN,NaN,"Phoenix, AZ",38060,2022-07-10 14:38:35,85232,Arizona,AZ,Pinal,Florence
92498,2019-02-01,2019-04-30,90,zip code,2,36283,f,AZ,All Residential,-1,...,NaN,NaN,"Phoenix, AZ",38060,2022-07-10 14:38:35,85232,Arizona,AZ,Pinal,Florence


# 3. Loading data into Power BI

Note: As Power BI will not allow us to retrieve more than 2gb of data using a python script. We will download our data into a .csv.

In [7]:
az_redfin_df.to_csv(r'C:\Users\Dan\Desktop\Coding\Datasets\Redfin_Data\Redfin_Zip_Realtor_Data.csv')